Due to this video, I need to experiment a bit.

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('spUNpyF58BY')

Let's make a sine wave, as in the video:

In [ ]:
import numpy as np

def make_sine(freq):
    """Returns (t, amp) of sine wave with input frequency."""
    dt = 1/(freq*20)
    t = 1 + np.arange(0, 4.5 + dt, dt)
    return t, (1 + np.cos(2 * np.pi * freq * t)) / 2

In [ ]:
t, amp = make_sine(3)

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
hv.Curve((t, amp), kdims='t', vdims='pressure')

Now, let's wrap this curve up in the manner suggested by the video. Each moment in time get's mapped to an angle and then the radius of that angle is the pressure from the above graph.

In [ ]:
def time2angle(t, cycles_per_sec):
    """Converts time to an angle (in rad), at cycles_per_sec rate."""
    return t * cycles_per_sec * 2 * np.pi

time2angle(t, 0.5)[0:10]

Let's check that the last value is roughly equal to what we expect at 0.5 turns per second (i.e. 2 seconds are one turn or 2 $\pi$ and thus 4.5 secondes are 2.25 * $2 \pi$).

In [ ]:
time2angle(t, 0.5)[-1], 2.25 * 2 * np.pi

Good, now let's move on to that graph.

In [ ]:
angles = time2angle(t, 0.5)
r = amp
x = r * np.cos(angles)
y = r * np.sin(angles)

hv.Curve((x, y))

Nice! Let's see if we can make this a little bit interactive.

In [ ]:
mappings = {}
for cycles_per_sec in np.linspace(0.2, 1.5, num=20):
    angles = time2angle(t, cycles_per_sec)
    r = amp
    x = r * np.cos(angles)
    y = r * np.sin(angles)
    mappings[cycles_per_sec] = hv.Curve((x, y))
hv.output(hv.HoloMap(mappings, kdims='cycles_per_sec'), holomap='scrubber')

Let's make a more finely explorable diagram.

In [ ]:
def plot_circle_mapping(cycles_per_sec):
    angles = time2angle(t, cycles_per_sec)
    r = amp
    x = r * np.cos(angles)
    y = r * np.sin(angles)
    center_of_mass = np.array([x.mean(), y.mean()]).reshape(1, -1)
    mapping_curve = hv.Curve((x, y)).opts(xlim=(-1, 1), ylim=(-1, 1)) * hv.Scatter(center_of_mass).opts(size=10, color='red')
    
    
    n_blocks = int(t.max() * cycles_per_sec ) 
    vlines = [hv.VLine(val).opts(color='gray') for val in np.arange(n_blocks) / cycles_per_sec if val > 0]
    time_curve = hv.Curve((t, amp), kdims='t', vdims='pressure') * hv.Overlay(vlines)
    return (mapping_curve + time_curve).cols(1)

hv.DynamicMap(plot_circle_mapping, kdims=['cycles_per_sec']).redim.range(cycles_per_sec=(0., 10.)).redim.step(cycles_per_sec=0.01)